In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# ターゲットURL
url = "https://suumo.jp/chintai/tokyo/sa_other_02/mansion/"

# ページ数（最大5ページ）
max_pages = 894

# 物件情報を格納するリスト
property_info_list = []

# 1秒待機
time.sleep(1)

# ページごとにスクレイピング

for page in range(1, max_pages + 1):
    page_url = f"{url}?page={page}"
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, "html.parser")

    # 物件情報を取得
    for item in soup.find_all(class_="cassetteitem"):
        property_info = {
            "物件名": item.find(class_="cassetteitem_content-title").text.strip(),
            "所在地": item.find(class_="cassetteitem_detail-col1").text.strip(),
            "交通": item.find(class_="cassetteitem_detail-col2").text.strip(),
            "築年数": item.find(class_="cassetteitem_detail-col3").text.strip(),
        }
        # 階、賃料、管理費、間取り、専有面積を別の行に追加
        for row in item.find_all(class_="js-cassette_link"):
            floor = row.find_all("td")[2].text.strip().replace("階", "")
            rent = row.find(class_="cassetteitem_price--rent").text.strip().replace("万円", "").replace("-", "0")
            admin_fee = row.find(class_="cassetteitem_price--administration").text.strip().replace("円", "").replace("万円", "").replace("-", "0")
            layout = row.find(class_="cassetteitem_madori").text.strip()
            area = row.find(class_="cassetteitem_menseki").text.strip().replace("m2", "")
            property_info.update({"階": floor, "賃料": float(rent) * 10000, "管理費": float(admin_fee), "間取り": layout, "専有面積": area})
            property_info_list.append(property_info.copy())  # コピーを追加

# データフレームに変換
df = pd.DataFrame(property_info_list)

# 修正後の DataFrame を表示
df


,物件名,所在地,交通,築年数,階,賃料,管理費,間取り,専有面積
0,近藤ビル,東京都台東区根岸５,東京メトロ日比谷線/三ノ輪駅 歩7分\nＪＲ山手線/鶯谷駅 歩16分\n東京メトロ日比谷線/...,築42年\n地下1地上4階建,3,115000.0,0.0,1LDK,54.02
1,ライフピア桜,東京都足立区千住桜木１,ＪＲ常磐線/北千住駅 歩12分\n京成本線/千住大橋駅 歩13分\n都電荒川線/荒川七丁目駅...,築91年\n7階建,6,65000.0,4500.0,1DK,23.04
2,三家第1ビル,東京都江戸川区中央３,ＪＲ総武線快速/新小岩駅 歩20分\nＪＲ総武線/小岩駅 歩30分\n都営新宿線/船堀駅 バ...,築61年\n7階建,4,56500.0,3000.0,ワンルーム,24.6
3,三家第1ビル,東京都江戸川区中央３,ＪＲ総武線快速/新小岩駅 歩20分\nＪＲ総武線/小岩駅 歩30分\n都営新宿線/船堀駅 バ...,築61年\n7階建,2,60000.0,3000.0,ワンルーム,23.26
4,都営新宿線 一之江駅 4階建 築59年,東京都江戸川区松江２,都営新宿線/一之江駅 歩25分\n都営新宿線/船堀駅 歩26分\n都営新宿線/瑞江駅 歩32分,築59年\n4階建,3,63000.0,0.0,2DK,27.89
...,...,...,...,...,...,...,...,...,...
36305,つくばエクスプレス 浅草駅 10階建 新築,東京都台東区西浅草２,つくばエクスプレス/浅草駅 歩1分\n東京メトロ銀座線/田原町駅 歩7分\n東武伊勢崎線/浅...,新築\n10階建,7,239000.0,15000.0,1LDK,46.28
36306,つくばエクスプレス 浅草駅 10階建 新築,東京都台東区西浅草２,つくばエクスプレス/浅草駅 歩1分\n東京メトロ銀座線/田原町駅 歩7分\n東武伊勢崎線/浅...,新築\n10階建,8,240000.0,15000.0,1LDK,46.28
36307,つくばエクスプレス 浅草駅 10階建 新築,東京都台東区西浅草２,つくばエクスプレス/浅草駅 歩1分\n東京メトロ銀座線/田原町駅 歩7分\n東武伊勢崎線/浅...,新築\n10階建,9,241000.0,15000.0,1LDK,46.28
36308,つくばエクスプレス 浅草駅 10階建 新築,東京都台東区西浅草２,つくばエクスプレス/浅草駅 歩1分\n東京メトロ銀座線/田原町駅 歩7分\n東武伊勢崎線/浅...,新築\n10階建,10,243000.0,15000.0,1LDK,46.28


In [8]:
import re

# 築年数の2行目を削除
df["築年数"] = df["築年数"].str.split("\n").str[0]

# 所在地から「東京都」の後から「○○区」までを抜き出す関数
def extract_district(location):
    match = re.search(r'東京都(.+?区)', location)
    if match:
        return match.group(1)
    else:
        return None

# 区という項目に出力
df['区'] = df['所在地'].apply(extract_district)

# 修正後の DataFrame を表示
df


,物件名,所在地,交通,築年数,階,賃料,管理費,間取り,専有面積,区
0,近藤ビル,東京都台東区根岸５,東京メトロ日比谷線/三ノ輪駅 歩7分\nＪＲ山手線/鶯谷駅 歩16分\n東京メトロ日比谷線/...,築42年,3,115000.0,0.0,1LDK,54.02,台東区
1,ライフピア桜,東京都足立区千住桜木１,ＪＲ常磐線/北千住駅 歩12分\n京成本線/千住大橋駅 歩13分\n都電荒川線/荒川七丁目駅...,築91年,6,65000.0,4500.0,1DK,23.04,足立区
2,三家第1ビル,東京都江戸川区中央３,ＪＲ総武線快速/新小岩駅 歩20分\nＪＲ総武線/小岩駅 歩30分\n都営新宿線/船堀駅 バ...,築61年,4,56500.0,3000.0,ワンルーム,24.6,江戸川区
3,三家第1ビル,東京都江戸川区中央３,ＪＲ総武線快速/新小岩駅 歩20分\nＪＲ総武線/小岩駅 歩30分\n都営新宿線/船堀駅 バ...,築61年,2,60000.0,3000.0,ワンルーム,23.26,江戸川区
4,都営新宿線 一之江駅 4階建 築59年,東京都江戸川区松江２,都営新宿線/一之江駅 歩25分\n都営新宿線/船堀駅 歩26分\n都営新宿線/瑞江駅 歩32分,築59年,3,63000.0,0.0,2DK,27.89,江戸川区
...,...,...,...,...,...,...,...,...,...,...
36305,つくばエクスプレス 浅草駅 10階建 新築,東京都台東区西浅草２,つくばエクスプレス/浅草駅 歩1分\n東京メトロ銀座線/田原町駅 歩7分\n東武伊勢崎線/浅...,新築,7,239000.0,15000.0,1LDK,46.28,台東区
36306,つくばエクスプレス 浅草駅 10階建 新築,東京都台東区西浅草２,つくばエクスプレス/浅草駅 歩1分\n東京メトロ銀座線/田原町駅 歩7分\n東武伊勢崎線/浅...,新築,8,240000.0,15000.0,1LDK,46.28,台東区
36307,つくばエクスプレス 浅草駅 10階建 新築,東京都台東区西浅草２,つくばエクスプレス/浅草駅 歩1分\n東京メトロ銀座線/田原町駅 歩7分\n東武伊勢崎線/浅...,新築,9,241000.0,15000.0,1LDK,46.28,台東区
36308,つくばエクスプレス 浅草駅 10階建 新築,東京都台東区西浅草２,つくばエクスプレス/浅草駅 歩1分\n東京メトロ銀座線/田原町駅 歩7分\n東武伊勢崎線/浅...,新築,10,243000.0,15000.0,1LDK,46.28,台東区


In [9]:
import gspread
from gspread_dataframe import set_with_dataframe

# Google Sheets の認証
gc = gspread.service_account(filename='spread-sheet-api-419202-59ee0bee5075.json')  # 認証情報ファイルのパスを指定

# スプレッドシートを開く（スプレッドシートの URL から ID を取得して指定）
spreadsheet_id = '1S6f9kJ7in52I8d7OxEQrZQCx80KgYoGmCJhVhloen0Q'
worksheet_name = '賃貸物件'
sh = gc.open_by_key(spreadsheet_id)
worksheet = sh.worksheet(worksheet_name)

# DataFrame をスプレッドシートに書き込む
set_with_dataframe(worksheet, df)
